In [1]:
import tensorflow

In [2]:
import dataiku
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import gensim.downloader as api
from sklearn.metrics import f1_score
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import Dot,Reshape,Flatten
from tensorflow.keras.layers import Dense
from keras import backend as K
from tensorflow.keras.layers import Dense,Dropout,Activation,Embedding,Conv1D
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
import tensorflow.keras.backend as K
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
import keras_tuner as kt
from keras.models import Sequential,Model
from keras.layers import Dropout
from keras_tuner.tuners import BayesianOptimization
from kerastuner import HyperParameters
from itertools import product
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay,f1_score
from tensorflow.keras import regularizers

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
`import kerastuner` is deprecated, please use `import keras_tuner`.


In [27]:
path_folder=dataiku.Folder("Match_Occupation_Susovan").get_path()
# path_of_excel=os.path.join(path_folder+"/data for occupation match.xlsx")
df1=pd.read_csv(path_folder+"/occupation_mismatch_final.csv")

In [28]:
df1.head()

,Unnamed: 0,V_OCC_DESC,V_OCC_SEC_DESC,V_EMPLOYER_NAME_x,V_BIOGRAPHY,LABEL
0,0,MANUFACTURING LABOURERS,MANUFACTURING,BEYONICS PRECISION (M) SDN BHD,[BIOGRAPHY] To be determined. [IDENTIFICATION]...,0
1,1,MANUFACTURING LABOURERS,MANUFACTURING,BEYONICS PRECISION (M) SDN BHD,[BIOGRAPHY] To be determined. [IDENTIFICATION]...,0
2,2,ENGINEERING PROFESSIONALS (INCLUDING ELECTROTE...,CONSTRUCTION,SRI ESBINA (M) SDN BHD,[BIOGRAPHY] Military Attaché to Netherlands (J...,0
3,3,MANUFACTURING LABOURERS,GAMBLING/BETTING ACTIVITIES/JUNKET /CASINO/GAM...,GENTING MALAYSIA BERHAD,[BIOGRAPHY] General Manager of Guizhou Provinc...,0
4,4,MANUFACTURING LABOURERS,GAMBLING/BETTING ACTIVITIES/JUNKET /CASINO/GAM...,GENTING MALAYSIA BERHAD,[BIOGRAPHY] Member of the Board of Directors o...,0


In [29]:
path_folder=dataiku.Folder("Match_Occupation_Susovan").get_path()
# path_of_excel=os.path.join(path_folder+"/data for occupation match.xlsx")
df2=pd.read_csv(path_folder+"/occupation_match_final.csv")

In [30]:
df2.head()

,Unnamed: 0,V_OCC_DESC,V_OCC_SEC_DESC,V_EMPLOYER_NAME_x,V_BIOGRAPHY,LABEL
0,0,ADMINISTRATIVE AND COMMERCIAL MANAGERS,"AGRICULTURE (CROPS & ANIMAL PRODUCTION), AQUAC...",KOPERASI TENUSU NEGERI PERAK,[BIOGRAPHY] Police officer. [IDENTIFICATION] T...,1
1,1,SENIOR GOVERNMENT OFFICIALS,PUBLIC ADMINISTRATION AND DEFENCE; COMPULSORY ...,KEMENTERIAN LUAR NEGERI,[BIOGRAPHY] Ambassador Extraordinary and Pleni...,1
2,2,SENIOR GOVERNMENT OFFICIALS,PUBLIC ADMINISTRATION AND DEFENCE; COMPULSORY ...,KEMENTERIAN LUAR NEGERI,[BIOGRAPHY] Ambassador to China (Oct 2023 - )....,1
3,3,MANAGING DIRECTORS AND CHIEF EXECUTIVES,CONSULTANCY ACTIVITIES,3E CONSULTING SDN BHD,[BIOGRAPHY] Director of a corporate service pr...,1
4,4,SENIOR GOVERNMENT OFFICIALS,PUBLIC ADMINISTRATION AND DEFENCE; COMPULSORY ...,KEM PEMBANGUNAN USAHAWAN DAN KOPERASI,[BIOGRAPHY] Deputy Secretary General (Policy a...,1


In [31]:
path_folder=dataiku.Folder("Match_Occupation_Susovan").get_path()
# path_of_excel=os.path.join(path_folder+"/data for occupation match.xlsx")
df3=pd.read_csv(path_folder+"/cosine.csv")

In [32]:
df3.head()

,Unnamed: 0,V_OCC_DESC,V_OCC_SEC_DESC,V_EMPLOYER_NAME_x,V_BIOGRAPHY,LABEL
0,1,MANAGING DIRECTORS AND CHIEF EXECUTIVES,FINANCIAL INSTITUITION,UNION BANK OF INDIA,[BIOGRAPHY] MEMBER OF THE BOARD OF DIRECTORS O...,1
1,2,MANAGING DIRECTORS AND CHIEF EXECUTIVES,FINANCIAL INSTITUITION,UNION BANK OF INDIA,[BIOGRAPHY] MEMBER OF THE BOARD OF DIRECTORS O...,1
2,27,MANAGING DIRECTORS AND CHIEF EXECUTIVES,FINANCIAL INSTITUITION,UNITED COMMERCIAL BANK LIMITED,[BIOGRAPHY] MEMBER OF THE BOARD OF DIRECTORS O...,1
3,32,MANAGING DIRECTORS AND CHIEF EXECUTIVES,FINANCIAL INSTITUITION,UNITED COMMERCIAL BANK LIMITED,[BIOGRAPHY] MEMBER OF THE BOARD OF DIRECTORS O...,1
4,51,SERVICES MANAGERS,HUMAN HEALTH AND SOCIAL WORK ACTIVITIES,COLUMBIA ASIA SDN BHD,[BIOGRAPHY] FORMER PROJECT DEVELOPMENT MANAGER...,1


In [43]:
df=pd.concat([df1,df2,df3],axis=0)

In [44]:
df.shape

(15789, 6)

In [35]:
import os
folder_path=dataiku.Folder('Match_Occupation_Susovan').get_path()
df.to_csv(os.path.join(folder_path,'occupation_final.csv'))

In [45]:
df[(df["V_OCC_DESC"].isnull()) & (df["V_OCC_SEC_DESC"].isnull()) & (df["V_EMPLOYER_NAME_x"].isnull())]

,Unnamed: 0,V_OCC_DESC,V_OCC_SEC_DESC,V_EMPLOYER_NAME_x,V_BIOGRAPHY,LABEL
20,20,NaN,NaN,NaN,[BIOGRAPHY] Member of the Jiangxi Provincial P...,1
21,21,NaN,NaN,NaN,[BIOGRAPHY] Member of the Jiangxi Provincial P...,1


In [46]:
df.drop(df[(df["V_OCC_DESC"].isnull()) & (df["V_OCC_SEC_DESC"].isnull()) & (df["V_EMPLOYER_NAME_x"].isnull())].index,axis=0,inplace=True)

In [47]:
df.reset_index(inplace=True)

In [48]:
df["CUSTOMER_BIOGRAPHY"]=["" for i in df.index]

In [49]:
df.isnull().sum()

index                   0
Unnamed: 0              0
V_OCC_DESC              3
V_OCC_SEC_DESC          3
V_EMPLOYER_NAME_x     430
V_BIOGRAPHY             0
LABEL                   0
CUSTOMER_BIOGRAPHY      0
dtype: int64

In [50]:
# if all three available
key=0
j=[]
for i in df.index:

    s=""
    if np.any(df.loc[i,["V_OCC_DESC"]].notnull()):
        s=df.loc[i,"V_OCC_DESC"]
#         print(s)
    if np.any(df.loc[i,["V_EMPLOYER_NAME_x"]].notnull()):
        if s!="":
            s=s+" "+"at"+" "+df.loc[i,'V_EMPLOYER_NAME_x']
            key=1 # employer & occ present
        else:
            s="employee "+"at"+" "+df.loc[i,'V_EMPLOYER_NAME_x']
            key=2 # employer & occ not present
    else:
         if s!="":
                key=3 # occ present but no employer but occ present
        
    if np.any(df.loc[i,["V_OCC_SEC_DESC"]].notnull()):
        if s=="": # no employer no occ present , dept present
            s=df.loc[i,"V_OCC_SEC_DESC"]+" sector"
        elif key==1 or key==2 or key==3:# occ may/may not be present but employer  present, key==3 occ present but employer not present
            s=s+", "+df.loc[i,"V_OCC_SEC_DESC"]+" sector"
     
    else:# no dept desc
        pass
            
    j.append(s) 


In [51]:
df["CUSTOMER_BIOGRAPHY"]=j
df['CUSTOMER_BIOGRAPHY']=df['CUSTOMER_BIOGRAPHY'].str.upper()


In [52]:
df['V_OCC_DESC']=df['V_OCC_DESC'].apply(lambda x: "" if pd.isna(x) else x)
df['V_EMPLOYER_NAME_x']=df['V_EMPLOYER_NAME_x'].apply(lambda x: "" if pd.isna(x) else x)
df["V_OCC_SEC_DESC"]=df["V_OCC_SEC_DESC"].apply(lambda x: "" if pd.isna(x) else x)


In [53]:
import calendar
cal=list(calendar.month_name)
for i in list(calendar.month_name[1:]):
    cal.append(i[:3])


In [54]:
cal_re="(%s)" % "|".join(cal)[1:]
cal_re

'(January|February|March|April|May|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)'

In [55]:
import re
# biography examination
patt=re.compile("\(sanctioned entity\)|\(reported "+cal_re+" \d{4} (-*) \)(\.{0,1})|"+"\(\s*"+cal_re+" \d{4} (-*) "+cal_re+" \d{4}\s*\)|\(Associated to sanctioned entity (-*) reported \d{4}(\s*\.*)\)|\(\s*-\s*"+cal_re+"\s*\d{4}\)\s*\(\s*-\s*"+cal_re+"\s*\d{4}\s*\)|To be determined(\s*\.*)|",re.IGNORECASE)
patt2=re.compile("REQUIRES ECDD & SENIOR MGMT APPROVAL|PNG Mule Accounts (WCL)|EXIT DUE TO ML CONCERN|NO ACCOUNT TO BE OPENED|AMLA FREEZING ORDER|AMLA FORFEITURE|CLOSE ACC AS ADVISED BY AG|Exit due to ML/TF concern|NO ACCOUNT TO BE OPENED \(OU\)",re.IGNORECASE)
print(re.sub(patt,"","(reported Aug 2019 - )"))
print(re.sub(patt,"","(reported Aug 2019 - )."))
print(re.sub(patt,"","(Aug 2020 - Oct 2022)"))
print(re.sub(patt,"","( - Jun 2018) ( - Jun 2018)"))
print(re.sub(patt,"","To be determined"))


invalid escape sequence \(
invalid escape sequence \d
invalid escape sequence \(
invalid escape sequence \d
invalid escape sequence \d
invalid escape sequence \s
invalid escape sequence \s
invalid escape sequence \(
invalid escape sequence \(
invalid escape sequence \d
invalid escape sequence \(
invalid escape sequence \d
invalid escape sequence \d
invalid escape sequence \s
invalid escape sequence \s
invalid escape sequence \(


In [56]:
df['V_BIOGRAPHY']=df['V_BIOGRAPHY'].str.upper()

In [57]:
df['WATCHLIST_BIOGRAPHY'] = df['V_BIOGRAPHY'].copy()
df["WATCHLIST_BIOGRAPHY"]=df["WATCHLIST_BIOGRAPHY"].apply(lambda x: re.sub(patt,"",x))
df["WATCHLIST_BIOGRAPHY"]=df["WATCHLIST_BIOGRAPHY"].apply(lambda x: re.sub(patt2,"",x))


In [58]:
# df['WATCHLIST_BIOGRAPHY'] = df['V_BIOGRAPHY'].str.extract(r'\[BIOGRAPHY\](.*?)\[IDENTIFICATION\]')  
ind1=df[df['WATCHLIST_BIOGRAPHY'].apply(lambda x: len(re.findall('\[BIOGRAPHY\](.*?)\[IDENTIFICATION\]',x))>0)].index
ind2=df[df['WATCHLIST_BIOGRAPHY'].apply(lambda x: len(re.findall('\[BIOGRAPHY\](.*?)\[IDENTIFICATION\]',x))==0)].index


invalid escape sequence \[
invalid escape sequence \[
invalid escape sequence \[
invalid escape sequence \[


In [59]:
# truncation for biographies containing biography secion
df.loc[ind1.to_list(),"WATCHLIST_BIOGRAPHY"]=df.loc[ind1.to_list(),"WATCHLIST_BIOGRAPHY"].str.extract(r'\[BIOGRAPHY\](.*?)\[IDENTIFICATION\]').values.flatten()

In [60]:
# Truncation for biographies where no biography secion is present  - 

df.loc[ind2.to_list(),"WATCHLIST_BIOGRAPHY"]=df.loc[ind2.to_list(),"WATCHLIST_BIOGRAPHY"].apply(lambda x:".".join(re.split("\.",x)[:3]))

invalid escape sequence \.
invalid escape sequence \.


In [61]:
df['WATCHLIST_BIOGRAPHY']=df['WATCHLIST_BIOGRAPHY'].apply(lambda x: "" if pd.isna(x) else x)
df['WATCHLIST_BIOGRAPHY']=df['WATCHLIST_BIOGRAPHY'].apply(lambda x: re.sub("^\s*","",x))
df=df[df["WATCHLIST_BIOGRAPHY"].apply(lambda x: x!="")]


invalid escape sequence \s
invalid escape sequence \s


In [63]:
# Load pre-trained BERT tokenizer and model
package = "/app/dataiku/design/code-envs/resources/python/py_39_tm_ns_2/bert-base-uncased"
package2 = "/app/dataiku/design/code-envs/resources/python/py_39_tm_ns_2/bert-large-cased"
tokenizer = BertTokenizer.from_pretrained(package)
model = BertModel.from_pretrained(package)


In [64]:
# Function to process a single pair of sentences and return the CLS token embedding
def process_pair(sentence1, sentence2):
    # Tokenize sentences and add special tokens
    tokens = tokenizer(sentence1, sentence2, return_tensors='pt', padding=True, truncation=True,max_length=512)
    
    # Pass tokenized input to BERT model to obtain CLS token embeddings
    with torch.no_grad():
        outputs = model(**tokens)
    
    # Extract CLS token embeddings
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    #print(cls_embedding)
    
    return cls_embedding


In [66]:
embeddings = []
for index, row in df.iterrows():
    sentence1 = row['CUSTOMER_BIOGRAPHY']
    sentence2 = row['WATCHLIST_BIOGRAPHY']
    cls_embedding = process_pair(sentence1, sentence2)
    embeddings.append(cls_embedding)

    
# Stack the embeddings into a single tensor
embeddings_tensor = torch.stack(embeddings)

# Add a new column to the DataFrame to store the embeddings
df['Embeddings'] = embeddings_tensor.tolist()


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [67]:
x = np.array(df['Embeddings'].tolist())

In [69]:
model = tf.keras.models.load_model(path_folder+'/bert30.hdf5',compile=False)

In [70]:
y_pred_proba=model.predict(x)

In [71]:
y_pred_proba = np.squeeze(y_pred_proba)

In [72]:
df['pred_proba']=y_pred_proba


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [73]:
df['pred_label'] = np.where(df['pred_proba'] >=0.6645332266613331, 1, 0)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [74]:
print(classification_report(df['LABEL'],df['pred_label']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



In [77]:
df=df[['V_OCC_DESC','V_OCC_SEC_DESC','V_EMPLOYER_NAME_x','V_BIOGRAPHY','LABEL','pred_label']]

In [0]:
df = df[['V_OCC_DESC', 'V_OCC_SEC_DESC', 'V_EMPLOYER_NAME_x', 'V_BIOGRAPHY', 'LABEL']]